In [1]:
import pandas as pd
import requests
pd.set_option('display.max_columns', 500)


In [88]:
all_lines = pd.read_csv('prize_picks_lines.csv')

In [89]:
def call_endpoint(url, max_level=3, include_new_player_attributes=False):
    '''
    takes: 
        - url (str): the API endpoint to call
        - max_level (int): level of json normalizing to apply
        - include_player_attributes (bool): whether to include player object attributes in the returned dataframe
    returns:
        - df (pd.DataFrame): a dataframe of the call response content
    '''
    resp = requests.get(url).json()
    data = pd.json_normalize(resp['data'], max_level=max_level)
    included = pd.json_normalize(resp['included'], max_level=max_level)
    if include_new_player_attributes:
        inc_cop = included[included['type'] == 'new_player'].copy().dropna(axis=1)
        data = pd.merge(data
                        , inc_cop
                        , how='left'
                        , left_on=['relationships.new_player.data.id'
                                   ,'relationships.new_player.data.type']
                        , right_on=['id', 'type']
                        , suffixes=('', '_new_player'))
    return data

In [90]:
url = 'http://partner-api.prizepicks.com/leagues' # leagues that are live on our board
print(url)
leagues_df = call_endpoint(url)

http://partner-api.prizepicks.com/leagues


In [91]:
leagues_df = leagues_df[['attributes.name', 'id', 'attributes.projections_count']]
leagues_df[leagues_df['attributes.projections_count'] > 0]

,attributes.name,id,attributes.projections_count
0,WNBA,3,25
9,NBA,7,608
13,MLB,2,1470
16,NHL,8,342
23,SOCCER,82,476
24,MLBLIVE,231,47
29,LoL,121,80
30,NBA1Q,192,19
31,NHL1P,227,21
35,COD,145,108


In [93]:
league_id = 2
url = f'https://partner-api.prizepicks.com/projections?league_id={league_id}&per_page=1000'
prop_df = call_endpoint(url, include_new_player_attributes=True)

In [94]:
simple_prop_df = prop_df.copy()

simple_prop_df = simple_prop_df[['attributes.start_time','attributes.name', 'type', 'id', 'attributes.adjusted_odds', 'attributes.line_score', 'attributes.odds_type', 'attributes.projection_type', 'attributes.stat_type', 'relationships.new_player.data.id','attributes.position', 'attributes.team', 'attributes.image_url']]
simple_prop_df = simple_prop_df.rename(columns={
    'attributes.start_time' : 'game_date',
    'attributes.line_score' : 'prop_line',
    'attributes.odds_type' : 'odds_type',
    'attributes.stat_type' : 'prop_type',
    'relationships.new_player.data.id' : 'PP_ID',
    'attributes.name' : 'Name',
    'attributes.image_url' : 'image_url',
    'attributes.position' : 'position',
    'attributes.team' : 'team_name'
})
simple_prop_df['game_date_time'] = pd.to_datetime(simple_prop_df['game_date'])
simple_prop_df['game_date'] = simple_prop_df['game_date_time'].dt.date


In [80]:


pitcher_props = ['Pitcher Fantasy Score', 'Hits Allowed', 'Pitching Outs', 'Pitcher Strikeouts', 'Earned Runs Allowed', 'Walks Allowed']

simple_prop_df[simple_prop_df['prop_type'].isin(pitcher_props)]

,game_date,Name,type,id,attributes.adjusted_odds,prop_line,odds_type,attributes.projection_type,prop_type,PP_ID,position,team_name,image_url,game_date_time
0,2024-05-10,Javier Assad,projection,2429798,None,15.5,standard,Single Stat,Pitching Outs,73818,P,CHC,https://static.prizepicks.com/images/players/m...,2024-05-10 18:40:00-04:00
1,2024-05-10,Javier Assad,projection,2429800,None,4.5,standard,Single Stat,Hits Allowed,73818,P,CHC,https://static.prizepicks.com/images/players/m...,2024-05-10 18:40:00-04:00
2,2024-05-10,Jared Jones,projection,2429389,None,6.5,standard,Single Stat,Pitcher Strikeouts,192811,P,PIT,https://static.prizepicks.com/images/players/p...,2024-05-10 18:40:00-04:00
3,2024-05-10,Framber Valdez,projection,2429175,None,33.5,standard,Single Stat,Pitcher Fantasy Score,67725,P,HOU,https://static.prizepicks.com/images/players/m...,2024-05-10 18:40:00-04:00
6,2024-05-10,Clarke Schmidt,projection,2429176,None,29.5,standard,Single Stat,Pitcher Fantasy Score,74520,P,NYY,https://static.prizepicks.com/images/players/m...,2024-05-10 18:50:00-04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,2024-05-10,Tyler Glasnow,projection,2429870,True,5.5,demon,Single Stat,Hits Allowed,188897,P,LAD,https://static.prizepicks.com/images/players/m...,2024-05-10 21:40:00-04:00
1090,2024-05-10,Bryan Woo,projection,2432674,True,3.5,demon,Single Stat,Hits Allowed,142204,P,SEA,https://static.prizepicks.com/images/players/p...,2024-05-10 21:40:00-04:00
1095,2024-05-10,Paul Blackburn,projection,2431141,True,4.5,goblin,Single Stat,Hits Allowed,67593,P,OAK,https://static.prizepicks.com/images/players/m...,2024-05-10 21:40:00-04:00
1099,2024-05-10,Andrew Abbott,projection,2429871,True,4.5,goblin,Single Stat,Hits Allowed,142294,P,CIN,https://static.prizepicks.com/images/players/p...,2024-05-10 22:15:00-04:00


In [95]:
all_lines = pd.concat([all_lines, simple_prop_df])
all_lines = all_lines.drop_duplicates(subset=['game_date_time', 'Name', 'prop_type', 'prop_line'], keep='first')


In [96]:
all_lines

,game_date,Name,type,id,attributes.adjusted_odds,prop_line,odds_type,attributes.projection_type,prop_type,PP_ID,position,team_name,image_url,game_date_time
0,2024-05-10,Framber Valdez,projection,2429175,NaN,33.5,standard,Single Stat,Pitcher Fantasy Score,67725,P,HOU,https://static.prizepicks.com/images/players/m...,2024-05-10 18:40:00-04:00
1,2024-05-10,Yordan Alvarez,projection,2429695,True,0.5,demon,Single Stat,Home Runs,67700,OF,HOU,https://static.prizepicks.com/images/players/m...,2024-05-10 18:40:00-04:00
2,2024-05-10,Jared Jones,projection,2429389,NaN,6.5,standard,Single Stat,Pitcher Strikeouts,192811,P,PIT,https://static.prizepicks.com/images/players/p...,2024-05-10 18:40:00-04:00
3,2024-05-10,Javier Assad,projection,2429798,NaN,15.5,standard,Single Stat,Pitching Outs,73818,P,CHC,https://static.prizepicks.com/images/players/m...,2024-05-10 18:40:00-04:00
4,2024-05-10,Nico Hoerner,projection,2429712,True,0.5,demon,Single Stat,Stolen Bases,69425,IF,CHC,https://static.prizepicks.com/images/players/m...,2024-05-10 18:40:00-04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,2024-05-10,Thairo Estrada,projection,2431456,True,0.5,goblin,Single Stat,Total Bases,71007,IF,SF,https://static.prizepicks.com/images/players/m...,2024-05-10 22:15:00-04:00
1466,2024-05-10,Thairo Estrada,projection,2431457,True,2.5,demon,Single Stat,Total Bases,71007,IF,SF,https://static.prizepicks.com/images/players/m...,2024-05-10 22:15:00-04:00
1467,2024-05-10,Tyler Fitzgerald,projection,2431458,True,1.5,demon,Single Stat,Total Bases,162059,IF,SF,https://static.prizepicks.com/images/players/p...,2024-05-10 22:15:00-04:00
1468,2024-05-10,Will Benson,projection,2431460,True,1.5,demon,Single Stat,Total Bases,127209,OF,CIN,https://static.prizepicks.com/images/players/m...,2024-05-10 22:15:00-04:00


In [ ]:
all_lines.to_csv('prize_picks_lines.csv', index=False)

In [85]:
league_id = 231
url = f'https://partner-api.prizepicks.com/projections?league_id={league_id}&per_page=1000'
prop_df = call_endpoint(url, include_new_player_attributes=True)

In [86]:
prop_df

,type,id,attributes.adjusted_odds,attributes.board_time,attributes.custom_image,attributes.description,attributes.end_time,attributes.flash_sale_line_score,attributes.game_id,attributes.hr_20,attributes.is_promo,attributes.line_score,attributes.odds_type,attributes.projection_type,attributes.rank,attributes.refundable,attributes.start_time,attributes.stat_type,attributes.status,attributes.today,attributes.tv_channel,attributes.updated_at,relationships.duration.data.type,relationships.duration.data.id,relationships.league.data.type,relationships.league.data.id,relationships.new_player.data.type,relationships.new_player.data.id,relationships.projection_type.data.type,relationships.projection_type.data.id,relationships.stat_type.data.type,relationships.stat_type.data.id,type_new_player,id_new_player,attributes.combo,attributes.display_name,attributes.image_url,attributes.league,attributes.league_id,attributes.market,attributes.name,attributes.position,attributes.team,attributes.team_name,relationships.league.data.type_new_player,relationships.league.data.id_new_player
0,projection,2428942,None,2024-05-09T18:40:00-04:00,None,HOU 1st Inning,None,None,2601b145-a19d-498f-8eea-5e721ba80776,True,False,16.0,standard,Single Stat,1,True,2024-05-10T18:40:00-04:00,Pitches Thrown,pre_game,True,None,2024-05-10T08:55:15-04:00,duration,279,league,231,new_player,193087,projection_type,2,stat_type,233,new_player,193087,False,Casey Mize,https://static.prizepicks.com/images/players/m...,MLBLIVE,231.0,Detroit,Casey Mize,P,DET,Tigers,league,231
1,projection,2428943,None,2024-05-09T18:40:00-04:00,None,DET 1st Inning,None,None,2601b145-a19d-498f-8eea-5e721ba80776,True,False,15.0,standard,Single Stat,2,True,2024-05-10T18:40:00-04:00,Pitches Thrown,pre_game,True,None,2024-05-10T08:55:15-04:00,duration,279,league,231,new_player,142422,projection_type,2,stat_type,233,new_player,142422,False,Framber Valdez,https://static.prizepicks.com/images/players/m...,MLBLIVE,231.0,Houston,Framber Valdez,P,HOU,Astros,league,231
2,projection,2428956,None,2024-05-09T18:40:00-04:00,None,PIT 1st Inning,None,None,e672d616-09d5-4d60-8c29-5e653fd97f4e,True,False,15.5,standard,Single Stat,21,True,2024-05-10T18:40:00-04:00,Pitches Thrown,pre_game,True,None,2024-05-10T08:55:15-04:00,duration,279,league,231,new_player,148484,projection_type,2,stat_type,233,new_player,148484,False,Javier Assad,https://static.prizepicks.com/images/players/m...,MLBLIVE,231.0,Chicago,Javier Assad,P,CHC,Cubs,league,231
3,projection,2428957,None,2024-05-09T18:40:00-04:00,None,CHC 1st Inning,None,None,e672d616-09d5-4d60-8c29-5e653fd97f4e,True,False,15.5,standard,Single Stat,22,True,2024-05-10T18:40:00-04:00,Pitches Thrown,pre_game,True,None,2024-05-10T08:55:15-04:00,duration,279,league,231,new_player,192812,projection_type,2,stat_type,233,new_player,192812,False,Jared Jones,https://static.prizepicks.com/images/players/p...,MLBLIVE,231.0,Pittsburgh,Jared Jones,P,PIT,Pirates,league,231
4,projection,2428964,None,2024-05-09T18:40:00-04:00,None,TB 1st Inning,None,None,533e7ade-0491-466f-82eb-e3d92238c5ec,True,False,15.5,standard,Single Stat,41,True,2024-05-10T18:50:00-04:00,Pitches Thrown,pre_game,True,None,2024-05-10T08:55:15-04:00,duration,279,league,231,new_player,127464,projection_type,2,stat_type,233,new_player,127464,False,Clarke Schmidt,https://static.prizepicks.com/images/players/m...,MLBLIVE,231.0,New York,Clarke Schmidt,P,NYY,Yankees,league,231
5,projection,2428989,None,2024-05-09T18:40:00-04:00,None,BAL 1st Inning,None,None,1f844014-ea96-4dd4-8e68-7686a9d8e16a,True,False,16.0,standard,Single Stat,61,True,2024-05-10T19:05:00-04:00,Pitches Thrown,pre_game,True,None,2024-05-10T08:55:15-04:00,duration,279,league,231,new_player,192910,projection_type,2,stat_type,233,new_player,192910,False,Brandon Pfaadt,https://static.prizepicks.com/images/players/m...,MLBLIVE,231.0,Arizona,Brandon Pfaadt,P,AZ,Diamondbacks,league,231
6,projection,2428990,None,2024-05-09T18:40:00-04:00,None,AZ 1st Inning,None,None,1f844